In [86]:
####################
# Volume des pav finalement non pris en compte... à supprimer?
# Rayon de couverture des pav fixé à 150m... pertinent ou pas?
# On pourrait tenir compte de leur volume et de la densité de population mais quels coefficients??
#################### 

In [87]:
import contextily as ctx
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import requests

from folium.plugins import GroupedLayerControl


# Préparation des données géographiques de Marseille

In [88]:
# Importation des données géographiques des quartiers de Marseille
url = 'https://www.data.gouv.fr/api/1/datasets/r/8a8f7f54-7f91-482c-a78c-dd09d893d1b6'
file = requests.get(url)
data = file.content
quartiers_data = gpd.read_file(data)
quartiers_data.head(2)

,DEPCO,NOM_CO,NOM_QUA,geometry
0,13201,Marseille 1er Arrondissemen,BELSUNCE,"MULTIPOLYGON (((5.38086 43.29924, 5.38087 43.2..."
1,13201,Marseille 1er Arrondissemen,CHAPITRE,"MULTIPOLYGON (((5.38525 43.29906, 5.38485 43.2..."


In [89]:
quartiers_data.loc[quartiers_data['NOM_QUA'].duplicated()]


,DEPCO,NOM_CO,NOM_QUA,geometry
30,13207,Marseille 7e Arrondissemen,ENDOUME,"MULTIPOLYGON (((5.34756 43.29063, 5.3489 43.28..."


In [90]:
quartiers_data = quartiers_data.drop_duplicates(subset = 'NOM_QUA', keep = 'first')

In [91]:
quartiers_map = quartiers_data.rename({'NOM_QUA' : 'Quartier'}, axis = 1)
quartiers_map.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [92]:
quartiers_map.loc[quartiers_map['Quartier'] == 'ENDOUME']

,DEPCO,NOM_CO,Quartier,geometry
29,13207,Marseille 7e Arrondissemen,ENDOUME,"MULTIPOLYGON (((5.34923 43.28452, 5.34919 43.2..."


# Préparation des données des pav

In [93]:
# Importation des données concernant les points d'apport volontaire
url = 'https://www.data.gouv.fr/api/1/datasets/r/e46c6879-49e7-4727-8f3b-62df90ac5a5a'
gouv_data = pd.read_csv(url, sep = ';')

In [94]:
gouv_data.head(2)

,OBJECTID,Ex identifiant CT,Date création mobilier,Famille Osis,Type de colonne,Modèle,Volume intérieur m3,longitude,latitude,Date de mise à jour de la donnée,...,Code postal,Quartier,Accessibilité,Nom de colonne,Abréviation type de flux,Trigramme de la commune,Propriétaire,created_date,last_edited_date,geo_point_2d
0,126496,PEY013,2019-01-01T00:00:00+00:00,Equipement Mobilier Urbain,Enterré,CONTENUR,5.0,5.552298,43.390463,2025-12-22T15:36:07+00:00,...,13124.0,NaN,OUI,NaN,O,PPN,Propriétaire Privé,2025-12-22T15:36:07+00:00,NaN,"43.39046332889413, 5.552297719881166"
1,126498,PEY013,2019-01-01T00:00:00+00:00,Equipement Mobilier Urbain,Enterré,CONTENUR,4.0,5.552257,43.390436,2025-12-22T15:36:07+00:00,...,13124.0,NaN,OUI,NaN,V,PPN,Propriétaire Privé,2025-12-22T15:36:07+00:00,NaN,"43.39043570369405, 5.5522573080508755"


In [95]:
# Extraction des variables d'intérêt
pav_data = gouv_data[['OBJECTID', 'Code Commune INSEE', 'Quartier', 'Volume intérieur m3', 'Type de flux', 'longitude', 'latitude']]


In [96]:
# Extraction des observations concernant les quartiers de Marseille
pav_marseille = pav_data.loc[(pav_data['Code Commune INSEE'] > 13200) & (pav_data['Code Commune INSEE'] < 13217)]
pav_marseille = pav_marseille.dropna(axis = 0, subset = ['Quartier'])
pav_marseille.head()


,OBJECTID,Code Commune INSEE,Quartier,Volume intérieur m3,Type de flux,longitude,latitude
20,126548,13201.0,SAINT CHARLES,2.2,Verre,5.390392,43.304831
21,126549,13201.0,SAINT CHARLES,3.2,Biflux,5.390395,43.304833
39,126606,13207.0,SAINT VICTOR,3.0,Biflux,5.368802,43.285029
40,126609,13201.0,THIERS,3.0,Verre,5.385471,43.296029
41,126611,13201.0,SAINT CHARLES,3.0,Biflux,5.393332,43.305732


In [97]:
# Uniformisation des noms de quartiers 
# Attention : certains quartiers de la carte ne sont pas référencés dans les données de pav (-> CARPIAGNE)

quartiers_map['Quartier'] = quartiers_map['Quartier'].replace(
    {
        'SAINT MAURON' : 'SAINT MAURONT',
        'VIELLE CHAPELLE' : 'VIEILLE CHAPELLE'
    }
)

pav_marseille['Quartier'] = pav_marseille['Quartier'].replace(
    {
    'CHATEAU GOMBERT' : 'CHATEAU-GOMBERT',
    'LA POINTE ROUGE' : 'POINTE ROUGE',
    'LA VILLETTE' : 'LA VILETTE',
    'LE ROUCAS' : 'ROUCAS BLANC',
    'LE ROUCAS BLANC' : 'ROUCAS BLANC',
    'LES CHUTES LAVIES' : 'CHUTES LAVIE',
    'LES CINQ AVENUES' : 'CINQ AVENUES',
    'LES GRANDS CARMES' : 'GRANDS CARMES',
    'ST BARNANE' : 'SAINT BARNABE',
    'STE MARGUERITE' : 'SAINTE MARGUERITE'
}
)




In [98]:
quartiers_map.loc[quartiers_map['Quartier'].duplicated()]

,DEPCO,NOM_CO,Quartier,geometry


In [99]:
# conversion en geodataframe
gdf_pav = gpd.GeoDataFrame(
    pav_marseille,
    geometry=gpd.points_from_xy(
        pav_marseille['longitude'],
        pav_marseille['latitude'],
        crs="EPSG:4326"))

gdf_pav = gdf_pav.to_crs(2154)

# Détermination de la zone couverte

In [100]:
zone_couverte = gdf_pav.copy()
zone_couverte['Volume intérieur m3'] = zone_couverte['Volume intérieur m3'].replace({2000.0 : 2.0, 3000.0 : 3.0})
zone_couverte['Volume intérieur m3'] = zone_couverte['Volume intérieur m3'].fillna(1.0)
zone_couverte['Volume intérieur m3'].unique()

array([2.2, 3.2, 3. , 4. , 1. , 2. , 5. , 1.8])

In [101]:
# Ajout des surfaces couvertes (rayon autour de chaque pav)
rayon = 150
zone_couverte['geometry'] = zone_couverte['geometry'].buffer(rayon)
zone_couverte.head(2)


,OBJECTID,Code Commune INSEE,Quartier,Volume intérieur m3,Type de flux,longitude,latitude,geometry
20,126548,13201.0,SAINT CHARLES,2.2,Verre,5.390392,43.304831,"POLYGON ((894183.444 6248011.361, 894182.721 6..."
21,126549,13201.0,SAINT CHARLES,3.2,Biflux,5.390395,43.304833,"POLYGON ((894183.655 6248011.572, 894182.933 6..."


In [102]:
types_flux = zone_couverte['Type de flux'].unique()
dict_gdf_flux = {}

for type in types_flux:
    dict_gdf_flux[type] = zone_couverte.loc[zone_couverte['Type de flux'] == type]


In [103]:
# regroupement de l'ensemble des zones de couverture en une seule zone couverte
# pour éviter la superposition de zones couvertes par plusieurs pav sur la carte finale

for type in types_flux:
    dict_gdf_flux[type] = dict_gdf_flux[type].dissolve().reset_index()

In [104]:
# Changement de système pour permettre les calculs d'aire
quartiers_map = quartiers_map.to_crs(2154)

In [105]:
# séparation de la zone couverte par quartier

dict_couverture = {}

for type in types_flux:
    dict_couverture[type] = quartiers_map.copy()
    dict_couverture[type]['geometry'] = dict_couverture[type]['geometry'].apply(lambda x : x.intersection(dict_gdf_flux[type]['geometry'][0]))
    
    # Calcul du taux de couverture par quartier
    dict_couverture[type]['taux_couverture'] = dict_couverture[type]['geometry'].area/quartiers_map['geometry'].area*100
    dict_couverture[type]['taux_couverture'] = dict_couverture[type]['taux_couverture'].apply(lambda x : round(x))

    dict_couverture[type] = dict_couverture[type].to_crs(3857)


In [109]:
for type in types_flux:
    quartiers_map = quartiers_map.merge(dict_couverture[type][['Quartier', 'taux_couverture']], on = 'Quartier', suffixes = [None, '_'+str(type)])

quartiers_map = quartiers_map.rename({'taux_couverture' : 'taux_couverture_Verre'}, axis = 1)

In [110]:
quartiers_map = quartiers_map.to_crs(3857)
gdf_pav = gdf_pav.to_crs(3857)

In [111]:
quartiers_map.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   DEPCO                       110 non-null    object  
 1   NOM_CO                      110 non-null    object  
 2   Quartier                    110 non-null    object  
 3   geometry                    110 non-null    geometry
 4   taux_couverture_Verre       110 non-null    int64   
 5   taux_couverture_Biflux      110 non-null    int64   
 6   taux_couverture_OM          110 non-null    int64   
 7   taux_couverture_Carton      110 non-null    int64   
 8   taux_couverture_Biodechets  110 non-null    int64   
 9   taux_couverture_Textile     110 non-null    int64   
dtypes: geometry(1), int64(6), object(3)
memory usage: 8.7+ KB


In [115]:
# fond de carte
map = folium.Map(
    location = [43.275, 5.4],
    tiles = 'cartodbpositron',
    zoom_start = 12,
    max_zoom=17,
    control_scale=True
    )



# ajout des couches pour chaque type de flux
layers = {}

for type in types_flux:
    layer_name = 'fg_'+ type 
    label = type
    layers[layer_name] = folium.FeatureGroup(name = label).add_to(map)


    dict_couverture[type].explore(
        m = layers[layer_name],
        cmap = 'Oranges',
        column = 'taux_couverture',
        popup = True,
        style_kwds =dict(fillOpacity = 0.8),
        legend = False
    )

    quartiers_map.explore(
        m = layers[layer_name],
        tooltip = ['Quartier', 'taux_couverture_'+str(type)],
        style_kwds = dict(
            fillOpacity = 0,
            color = 'black',
            opacity = 0.5),
        highlight_kwds = dict(
            fillOpacity = 0.5)
    )



    
    map.add_child(layers[layer_name])





# Contrôle de l'affichage des couches
GroupedLayerControl(
    groups={'Type de flux': [layer for layer in layers.values()]},
    collapsed=False,
).add_to(map)



map_title = "taux de couverture (avec un rayon de 150m par pav)"
title_html = f'<h1 align="center" style="font-size:24px" >{map_title}</h1>'
map.get_root().html.add_child(folium.Element(title_html))


map.save('../cartes/couverture_quartier.html')
